SELLAR'S PROBLEM 
----------------
This is a coupled disciplinary problem in OpenMDAO and has two disciplines. The problem can be stated as follows:

minimize  $x_1^2 +z_2 + y_1 + e^{-y_2}$

w.r.t :   $z_1 , z_2 , x_1$

subject to : $\frac{y_1}{3.16} - 1 \geq 0 $

$1-\frac{y_2}{24} \geq 0 $

$ -10 \leq z_1 \leq 10 $

$   0 \leq z_2 \leq 10 $

$   0 \leq x_1 \leq 10 $


Discipline 1 : $y_1 (z_1,z_2,x_1,y_2) = z_1 ^2 + x_1 + z_2 - 0.2y_2$ 

Discipline 2 : $y_2 (z_1,z_2,y_1) = \sqrt{y_1} + z_1  +  z_2 $ 


$z_1$ and $ z_2 $ are the global design variables while $x_1$ is a local design variable. $y_1$ and $y_2$ are coupling variables between then two disciplines.

This coupling creates a non-linear system of equations which must be satisfied for valid solutions.

TYPES OF MDO ARCHITECTURES
------------------------

 +   Multidisciplinary Design Feasible (MDF)
 
 In this architecture the disciplines are directly coupled via some kind of solver, and the design variables are optimized all at the top level.
<img src="files/Arch-MDF.png">


 + Individual Design Feasible (IDF)
  
  In IDF, the direct coupling between the disciplines is removed, and the input coupling variables are added to the optimizer’s design variables. The algorithm calls for two new equality constraints that enforce the coupling between the disciplines.
  
  <img src="files/Arch-IDF.png">
  
  
  + Collaborative Optimization (CO)
  
  CO is a two-level architecture with three optimizer loops, one at each discipline, and one acting globally. The global optimizer drives the design and coupling variables towards an optimal solution that minimizes the objective while constraining to zero the sum of the squares of the residuals between the values commanded by the global optimizer and those set by the local optimizers. Each local optimizer operates on its own discipline, driving its design variables while minimizing the residual between the actual value of the design variables and the values commanded by the global optimizer.
  
  <img src="files/Arch-CO.png">
  
  
  
  